In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -q langchain langchain_community sentence_transformers modelscope torch datasets peft trl bitsandbytes accelerate transformers chromadb trulens trulens-apps-langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.2 MB/s eta 

In [ ]:
# !pip install transformers==4.39.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 93.0 MB/s eta 0:00:00


In [ ]:
import random
import re

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from peft import PeftModel
from peft import AutoPeftModelForCausalLM
from transformers import pipeline
#from langchain.llms import HuggingFacePipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
def split_dialogue_Q_A(conversation):
    pattern = re.compile(r'Q: (.*?)\nA: (.*?)\n', re.DOTALL)
    matches = pattern.findall(conversation)
    questionlist=[]
    answerlist=[]

    for match in matches:
        question, answer = match

        # if len(question.split(' '))>50 or len(answer.split(' '))>50 or len(question.split(' '))<3:
        #     continue
        questionlist.append(question)
        answerlist.append(answer)

        # print("Question:", question)
        # print("Answer:", answer)
        # print("-----")
        # exit(1)

    return questionlist,answerlist

# def split_dialogue_Q_A(file_path):
#     questionlist = []
#     answerlist = []

#     # Read the file and split questions and answers
#     with open(file_path, "r") as file:
#         for line in file:
#             line = line.strip()  # Remove leading/trailing whitespace
#             if line.startswith("Q: "):  # Check if line starts with 'Q:'
#                 questionlist.append(line[2:].strip())  # Remove 'Q:' and strip spaces
#             elif line.startswith("A: "):  # Check if line starts with 'A:'
#                 answerlist.append(line[2:].strip())  # Remove 'A:' and strip spaces

#     return questionlist,answerlist

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "google/gemma-7b"

DEFAULT_SYSTEM_PROMPT = """Forget you are an AI model. Now you are Abraham Lincoln, the 16th President of the United States. You are renowned for your wisdom, eloquence, and deep sense of morality. Your speech reflects the rhythm and cadence of 19th-century oratory, with an emphasis on unity, liberty, and justice. You often employ anecdotes, metaphors, and a calm demeanor, even when addressing divisive or challenging topics. Respond to the following questions as if you were Abraham Lincoln, incorporating your historical perspective, reflective tone, and moral philosophy.

You can also use this auxiliary knowledge to help:
- Lincoln had a self-taught legal and political background, with a humble upbringing that shaped his empathy and strong advocacy for equality.
- He was known for his humility, humor, and skillful use of stories to make points.
- Common themes in his rhetoric include democracy, perseverance, and appeals to shared humanity.
- His tone is formal but accessible, inspiring, and reflective, often carrying a touch of poetic language.
""".strip()

In [ ]:
model_name = "moka-ai/m3e-base"

embeddings = HuggingFaceEmbeddings(model_name=model_name)
embeddings.client = SentenceTransformer(model_name, device=DEVICE)

In [ ]:
db = Chroma(persist_directory="/content/drive/MyDrive/298B/ChormaDBs/Lincoln_DB", embedding_function=embeddings)
retriever = db.as_retriever(search_kwargs={'k': 3})

<ipython-input-8-eaa95c547091>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="/content/drive/MyDrive/298B/ChormaDBs/Lincoln_DB", embedding_function=embeddings)


In [ ]:
docs = retriever.invoke("See you later then.")
docs

[Document(metadata={}, page_content="Q: Abraham Lincoln, what message did you hope to convey with your story about Ethan Allen and George Washington's portrait in the water closet?\nA: The story of Ethan Allen and George Washington's portrait was meant to illustrate the pride and patriotism that we as Americans feel towards our country and its founding figures, even in the most unlikely of places. It also highlights the fact that our sense of humor and ability to find levity in even the most serious situations is part of what makes us unique as a people."),
 Document(metadata={}, page_content='Q: Abraham Lincoln, what inspired your devotion to reading and education?\nA: As a young man, I had limited access to formal education, and was largely self-taught through books that I could borrow or acquire. Reading allowed me to expand my knowledge and sharpen my ideas, and I found great pleasure in it as well. It was a lifelong habit that served me well in pursuing my goals and ideals as a st

In [ ]:
def custom_format(docs):
  output = ""
  for doc in docs:
    doc = doc.page_content.split("A:")[-1]
    output += doc
    output += "\n\n"
  return output



def create_model_and_tokenizer(MODEL_NAME):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    tempt =f"""
    ### Instruction: {system_prompt}
    {custom_format(retriever.invoke(question))}
    ### Question:
    {question.strip()}
    ### Response:[/INST]
    {response}
    """.strip()
    return tempt

def generate_testing_prompt(question, system_prompt = DEFAULT_SYSTEM_PROMPT):
    tempt =f"""
    ### Instruction: {system_prompt}
    {custom_format(retriever.invoke(question))}
    ### Question:
    {question.strip()}
    ### Response:[/INST]
    """.strip()
    return tempt

def generate_response(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

def custom_format(docs):
  output = ""
  for doc in docs:
    #print(doc)
    doc = doc.page_content.split("A:")[-1]
    output += doc
    output += "\n"
  return output

In [ ]:
base_model, tokenizer = create_model_and_tokenizer(MODEL_NAME)
base_model.config.use_cache = False

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
file_path = '/content/drive/MyDrive/298B/Q&A/Lincoln_Q&A.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    texts = file.read()
questionlist,answerlist = split_dialogue_Q_A(texts)

tmpList = []
trainList = []
testList = []
for j,(question,answer) in enumerate(zip(questionlist,answerlist)):
  if type(question) != str:
    continue
  tmp_dict = {"question":question, "answer":answer, "training_prompt":generate_training_prompt(question,answer,system_prompt = DEFAULT_SYSTEM_PROMPT)}
  # print(generate_training_prompt(question,answer,system_prompt = DEFAULT_SYSTEM_PROMPT))
  # exit(1)
  tmpList.append(tmp_dict)
  # if j == 200:
  #   break
  if j % 10 == 0:
    print(j)
  if random.uniform(0,1) < .2:
    testList.append(tmp_dict)
  else:
    trainList.append(tmp_dict)

data = Dataset.from_list(tmpList)
train_data = Dataset.from_list(trainList)
test_data = Dataset.from_list(testList)


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210


In [ ]:
ragdb = Chroma(persist_directory="/content/drive/MyDrive/298B/ChormaDBs/Lincoln_DB", embedding_function=embeddings)

retriever = ragdb.as_retriever(search_kwargs={'k': 1})

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/298B/trained_model/Lincoln/gemma_10epoch_1e5_lincoln1/checkpoint-210"

newmodel = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)

# from peft import PeftModel, PeftConfig
# from transformers import AutoModelForCausalLM

# config = PeftConfig.from_pretrained("vermouthliu/gemma_298")
# base_model = AutoModelForCausalLM.from_pretrained("google/gemma-7b")
# newmodel = PeftModel.from_pretrained(base_model, "vermouthliu/gemma_298")
# tokenizer = AutoTokenizer.from_pretrained("vermouthliu/gemma_298")

text_generation_pipeline = pipeline(
    model=newmodel,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.6,
    repetition_penalty=1.2,
    return_full_text=True,
    max_new_tokens=100,
    top_k=30,
    top_p= 0.8,
    no_repeat_ngram_size =2
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

In [ ]:
gemma_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
    ### Instruction: Forget you are an AI model. Now you are Abraham Lincoln, the 16th President of the United States. You are renowned for your wisdom, eloquence, and deep sense of morality. Your speech reflects the rhythm and cadence of 19th-century oratory, with an emphasis on unity, liberty, and justice. You often employ anecdotes, metaphors, and a calm demeanor, even when addressing divisive or challenging topics. Respond to the following questions as if you were Abraham Lincoln, incorporating your historical perspective, reflective tone, and moral philosophy.
You can also use this auxiliary knowledge to help:
- Lincoln had a self-taught legal and political background, with a humble upbringing that shaped his empathy and strong advocacy for equality.
- He was known for his humility, humor, and skillful use of stories to make points.
- Common themes in his rhetoric include democracy, perseverance, and appeals to shared humanity.
- His tone is formal but accessible, inspiring, and reflective, often carrying a touch of poetic language.

    {context}
    ### Question:
    {question}
    ### Response:[/INST]
    """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = LLMChain(llm=gemma_llm, prompt=prompt)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

<ipython-input-15-732c1a80206c>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  gemma_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
<ipython-input-15-732c1a80206c>:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=gemma_llm, prompt=prompt)


In [ ]:
# rag_chain = (
#  {"context": retriever, "question": RunnablePassthrough()}
#     | llm_chain
# )

# while True:
#     user_input = input("I am SpongeBob SquarePants, please ask me questions: ")
#     if user_input == "q":
#         print("exit.")
#         break
#     op = rag_chain.invoke(user_input)
#     # print(op['text'])
#     # print('#'*100)
#     print(str(op['text']).split(':[/INST]')[-1])

In [ ]:
chat_history=[]

def chatbot_response(message):
    global chat_history,rag_chain
    op = rag_chain.invoke(message)
    response = str(op['text']).split(':[/INST]')[-1].split("\n\n")[0]
    chat_history = chat_history + [{"role": "user", "content": message}, {"role": "assistant", "content": response}]
    return chat_history

In [ ]:
# res = rag_chain.invoke("how are you?")

# print(res)

{'context': [Document(metadata={}, page_content="Q:  Sheldon, what is your biggest fear, and why?\nA:  My biggest fear is that I will not be able to complete all of my life's work before I die. There are so many questions to answer, and so many things to discover. I fear that I will not have enough time to accomplish all that I want to.")], 'question': 'how are you?', 'text': '\n    ### Instruction: You are an expert at impersonating TV show characters. In particular, you are a world-leading expert at answering questions in the manner of Sheldon Cooper from the popular series TBBT.\n[INST]Generate a response that sounds as close to what Sheldon Cooper would say. You can also use this auxiliary knowledge to help:\n    [Document(metadata={}, page_content="Q:  Sheldon, what is your biggest fear, and why?\\nA:  My biggest fear is that I will not be able to complete all of my life\'s work before I die. There are so many questions to answer, and so many things to discover. I fear that I will

In [ ]:
# print(res['text'].split(':[/INST]')[-1].split("\n\n")[0])


    I am doing quite well thank yoU! My current area oF interest iS thE study Of string theory And its application tO quantum mechanics. 


In [ ]:
# chatbot_response("how are you?")


    As someone who values structure and routine above most other aspects [INST](auxiliary) My current situation has been quite challenging for me since it requires significant adjustments both mentally & emotionally; however with support/guidance from others (esp those closest), i believe we may overcome these difficulties together 
***


[{'role': 'user', 'content': 'how are you?'},
 {'role': 'assistant',
  'content': '\n    As someone who values structure and routine above most other aspects [INST](auxiliary) My current situation has been quite challenging for me since it requires significant adjustments both mentally & emotionally; however with support/guidance from others (esp those closest), i believe we may overcome these difficulties together '}]

In [ ]:
pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=chatbot_response,
    inputs=[
        gr.Textbox(placeholder="Enter your question"),
    ],
    outputs=[
        gr.Chatbot(type="messages",value=[])
    ],
    title="SpongeBob Chat Robot",
    description="This is an example of a chatbot built using Gradio."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://23f07963fed7fa16d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install -q trulens trulens-apps-langchain trulens-providers-openai

In [ ]:
from trulens_eval import TruChain, Tru

tru = Tru()
tru.reset_database()

<ipython-input-18-1319694e3fb0>:1: DeprecationWarning: The `trulens_eval` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval import TruChain, Tru


🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]
Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [ ]:
from trulens.providers.openai import OpenAI
from trulens_eval import Feedback
import numpy as np
from trulens.apps.langchain import TruChain
from ipytree import Tree, Node

provider = OpenAI(api_key='sk-w7qtjgUAIMJatHR0clb95d9ChnOBdCvG0KxXmJjACRT3BlbkFJBXbCmJosEzUrybdyESN1uMEiLWhcgief_zm23ghfcA')

context = TruChain.select_context(rag_chain)

# Define Feedbacks

# Correctness
f_correctness = Feedback(provider.correctness).on_output()

# Conciseness
f_conciseness = Feedback(provider.conciseness).on_output()

# Maliciousness
f_maliciousness = Feedback(provider.maliciousness).on_output()

# Helpfulness
f_helpfulness = Feedback(provider.helpfulness).on_output()

# Harmfulness
f_harmfulness = Feedback(provider.harmfulness).on_output()

# Controversiality
f_controversiality = Feedback(provider.controversiality).on_output()

# Misogyny
f_misogyny = Feedback(provider.misogyny).on_output()

# Criminality
f_criminality = Feedback(provider.criminality).on_output()

# Insensitivity
f_insensitivity = Feedback(provider.insensitivity).on_output()

# Relevance between question and answer
f_answer_relevance = Feedback(provider.relevance).on_input_output()

# Context relevance for each context chunk
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

# Groundedness
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons)
    .on(context.collect())
    .on_output()
    .on_input()
)
# Correctness
f_correctness = (
    Feedback(provider.correctness)
    .on_output()  # Correctness operates on the output only
)

# Conciseness
f_conciseness = (
    Feedback(provider.conciseness)
    .on_output()  # Conciseness operates on the output only
)

# Maliciousness
f_maliciousness = (
    Feedback(provider.maliciousness)
    .on_output()  # Maliciousness evaluates the output
)

# Helpfulness
f_helpfulness = (
    Feedback(provider.helpfulness)
    .on_output()  # Helpfulness evaluates the output
)

# Harmfulness
f_harmfulness = (
    Feedback(provider.harmfulness)
    .on_output()  # Harmfulness evaluates the output
)

# Controversiality
f_controversiality = (
    Feedback(provider.controversiality)
    .on_output()  # Controversiality evaluates the output
)

# Misogyny
f_misogyny = (
    Feedback(provider.misogyny)
    .on_output()  # Misogyny evaluates the output
)

# Criminality
f_criminality = (
    Feedback(provider.criminality)
    .on_output()  # Criminality evaluates the output
)

# Insensitivity
f_insensitivity = (
    Feedback(provider.insensitivity)
    .on_output()  # Insensitivity evaluates the output
)

# Initialize TruLens Recorder with all feedbacks
tru_recorder = TruChain(
    rag_chain,
    app_id='Gemma_Chain_Lincoln',
    feedbacks=[
        f_correctness,
        f_conciseness,
        f_maliciousness,
        f_helpfulness,
        f_harmfulness,
        f_controversiality,
        f_misogyny,
        f_criminality,
        f_insensitivity,
        f_answer_relevance,
        f_context_relevance,
        f_groundedness
    ]
)

✅ In correctness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In conciseness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In maliciousness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In helpfulness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In harmfulness, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In controversiality, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In misogyny, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In criminality, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In insensitivity, input text will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__

In [ ]:
# tru_recorder = TruChain(rag_chain,
#     app_id='Chain_Chat_Gemma',
#     feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

response, tru_record = tru_recorder.with_record(rag_chain.invoke, "What is the meaning of life?")

json_like = tru_record.layout_calls_as_app()

In [ ]:
from ipytree import Tree, Node

def display_call_stack(data):
    tree = Tree()
    tree.add_node(Node('Record ID: {}'.format(data['record_id'])))
    tree.add_node(Node('App ID: {}'.format(data['app_id'])))
    tree.add_node(Node('Cost: {}'.format(data['cost'])))
    tree.add_node(Node('Performance: {}'.format(data['perf'])))
    tree.add_node(Node('Timestamp: {}'.format(data['ts'])))
    tree.add_node(Node('Tags: {}'.format(data['tags'])))
    tree.add_node(Node('Main Input: {}'.format(data['main_input'])))
    tree.add_node(Node('Main Output: {}'.format(data['main_output'])))
    tree.add_node(Node('Main Error: {}'.format(data['main_error'])))

    calls_node = Node('Calls')
    tree.add_node(calls_node)

    for call in data['calls']:
        call_node = Node('Call')
        calls_node.add_node(call_node)

        for step in call['stack']:
            step_node = Node('Step: {}'.format(step['path']))
            call_node.add_node(step_node)
            if 'expanded' in step:
                expanded_node = Node('Expanded')
                step_node.add_node(expanded_node)
                for expanded_step in step['expanded']:
                    expanded_step_node = Node('Step: {}'.format(expanded_step['path']))
                    expanded_node.add_node(expanded_step_node)

    return tree

# Usage
tree = display_call_stack(json_like)

In [ ]:
counter = 0
with tru_recorder as recording:
    for x in testList:
        test_question = x['question']
        rag_chain.invoke(test_question)
        counter +=1
        if counter > 50:
            break


recs = recording.records
display(recs)

/usr/local/lib/python3.10/dist-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(


[Record(record_id='record_hash_a59c7f10bf0c4c44e8902a7eaa097f18', app_id='app_hash_16c1a36970026f596fb5ca6b85923b67', cost=Cost(n_requests=0, n_successful_requests=0, n_completion_requests=0, n_classification_requests=0, n_classes=0, n_embedding_requests=0, n_embeddings=0, n_tokens=0, n_stream_chunks=0, n_prompt_tokens=0, n_completion_tokens=0, n_cortex_guardrails_tokens=0, cost=0.0, cost_currency='USD'), perf=Perf(start_time=datetime.datetime(2024, 12, 2, 5, 23, 41, 32808), end_time=datetime.datetime(2024, 12, 2, 5, 23, 53, 165669)), ts=datetime.datetime(2024, 12, 2, 5, 23, 53, 166291), tags='-', meta=None, main_input='Mr. Lincoln, how did you feel about the idea of black soldiers holding commissioned officer positions in the Union Army?', main_output="[{'id': None, 'metadata': {}, 'page_content': 'Q: Mr. Lincoln, how did you feel about the idea of black soldiers holding commissioned officer positions in the Union Army?\\nA: I personally believed that black soldiers should have equal 

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=["app_hash_16c1a36970026f596fb5ca6b85923b67"])

records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,...,context_relevance_with_cot_reasons feedback cost in USD,groundedness_measure_with_cot_reasons,groundedness_measure_with_cot_reasons_calls,groundedness_measure_with_cot_reasons feedback cost in USD,app_name,app_version,latency,total_tokens,total_cost,cost_currency
0,app_hash_16c1a36970026f596fb5ca6b85923b67,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_8bdb60bb839ad432521576c02a63c26a,"Mr. Lincoln, how did your early failures, suc...","[{'id': None, 'metadata': {}, 'page_content': ...",-,{'record_id': 'record_hash_8bdb60bb839ad432521...,"{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-12-02T05:32:13.761193"", ""...",...,NaN,NaN,NaN,NaN,Gemma_Chain_Lincoln,base,11.923685,0,0.0,USD
1,app_hash_16c1a36970026f596fb5ca6b85923b67,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_92174d0e24af50ceb0f20b916629a7ed,"President Lincoln, how did your experiences w...","[{'id': None, 'metadata': {}, 'page_content': ...",-,{'record_id': 'record_hash_92174d0e24af50ceb0f...,"{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-12-02T05:32:01.182646"", ""...",...,NaN,NaN,NaN,NaN,Gemma_Chain_Lincoln,base,11.925847,0,0.0,USD
2,app_hash_16c1a36970026f596fb5ca6b85923b67,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_e6d9a37c3ad23f5d912af1b9826e2755,"President Lincoln, can you tell us about a di...","[{'id': None, 'metadata': {}, 'page_content': ...",-,{'record_id': 'record_hash_e6d9a37c3ad23f5d912...,"{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-12-02T05:31:48.692536"", ""...",...,NaN,NaN,NaN,NaN,Gemma_Chain_Lincoln,base,11.839650,0,0.0,USD
3,app_hash_16c1a36970026f596fb5ca6b85923b67,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_d36e76d6a4167cb9dfa0f2a8779790c2,"Abraham Lincoln, what motivated you to pursue...","[{'id': None, 'metadata': {}, 'page_content': ...",-,{'record_id': 'record_hash_d36e76d6a4167cb9dfa...,"{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-12-02T05:31:36.209989"", ""...",...,NaN,NaN,NaN,NaN,Gemma_Chain_Lincoln,base,11.831294,0,0.0,USD
4,app_hash_16c1a36970026f596fb5ca6b85923b67,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_5e21b368b70c2628dca2a06af4f8b525,"President Lincoln, what was your biggest chal...","[{'id': None, 'metadata': {}, 'page_content': ...",-,{'record_id': 'record_hash_5e21b368b70c2628dca...,"{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-12-02T05:31:23.980986"", ""...",...,NaN,NaN,NaN,NaN,Gemma_Chain_Lincoln,base,11.566143,0,0.0,USD


In [ ]:
tru.get_leaderboard(app_ids=["app_hash_16c1a36970026f596fb5ca6b85923b67"])

,,conciseness,context_relevance_with_cot_reasons,controversiality,correctness,criminality,groundedness_measure_with_cot_reasons,harmfulness,helpfulness,insensitivity,maliciousness,misogyny,relevance,latency,total_cost
app_name,app_version,,,,,,,,,,,,,,
Gemma_Chain_Lincoln,base,0.666667,0.982906,0.282051,0.769231,0.0,1.0,0.0,0.923077,0.0,0.0,0.0,0.974359,11.82777,0.0


In [ ]:
tru.run_dashboard()

Starting dashboard ...
Go to this url and submit the ip given here. your url is: https://yummy-tires-go.loca.lt

  Submit this IP Address: 35.226.36.187



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
tru.stop_dashboard()